In [1]:
import yfinance as yf  # api yfinance to get historical price and big firm recommendation
import pandas as pd
import datetime
import numpy as np
import re

In [26]:
enddate = datetime.date.today()
startdate = enddate - datetime.timedelta(days = 10)

In [33]:
ticker = 'MARA'

In [34]:
data = yf.download(ticker, group_by="Ticker", start=startdate.strftime("%Y-%m-%d"), end=enddate.strftime("%Y-%m-%d"))
data['ticker'] = ticker

[*********************100%***********************]  1 of 1 completed


In [35]:
data

,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,
2021-04-27,35.099998,35.400002,33.779999,34.119999,34.119999,12994300,MARA
2021-04-28,34.180000,37.919998,33.580002,37.639999,37.639999,20558200,MARA
2021-04-29,36.099998,36.639999,33.799999,34.259998,34.259998,15562100,MARA
2021-04-30,34.980000,38.549999,34.669998,36.779999,36.779999,21980200,MARA
2021-05-03,38.200001,39.680000,35.209999,36.220001,36.220001,17671200,MARA
2021-05-04,34.880001,35.240002,32.220001,33.950001,33.950001,14261900,MARA
2021-05-05,34.500000,36.230000,32.950001,33.549999,33.549999,15077600,MARA
2021-05-06,33.590000,34.110001,30.000000,31.400000,31.400000,13870600,MARA


In [40]:
def generate_full_date(sdate, edate):
    dates = []
    while sdate < edate:
        dates.append(datetime.datetime.strftime(sdate,'%Y-%m-%d'))
        sdate += datetime.timedelta(days=1)
    return dates

In [42]:
dates = generate_full_date(startdate, enddate)
ticker = 'RIOT'
df = yf.download(ticker, group_by="Ticker", start=startdate.strftime("%Y-%m-%d"), end=enddate.strftime("%Y-%m-%d"))
df['ticker'] = ticker
date_index = df.index.strftime('%Y-%m-%d').tolist()
df_all_date = {'Date': ['']*10, 'Open': ['']*10, 'Close': ['']*10, 'Ticker': ['']*10}

for i, date in enumerate(dates):
    df_all_date['Date'][i] = date
    df_all_date['Ticker'][i] = df['ticker'][i]
    if date in date_index:
        df_all_date['Open'][i] = df['Open'][date]
        df_all_date['Close'][i] = df['Close'][date]
    else:
        df_all_date['Open'][i] = 0
        df_all_date['Close'][i] = 0

for i, date in enumerate(dates):
    if df_all_date['Open'][i] == 0:
        k = 0
        while df_all_date['Open'][i + k] == 0:
            k += 1
        s = df_all_date['Close'][i - 1]
        t = df_all_date['Open'][i + k] - s
        for j in range(k):
            df_all_date['Open'][i + j] = s + t/(2*k+1)*(2*j+1)
            df_all_date['Close'][i + j] = s + t/(2*k+1)*(2*j+2)

[*********************100%***********************]  1 of 1 completed


In [49]:
df_final = {'Date': ['']*20, 'Type': ['']*20, 'StockPrice': ['']*20, 'Ticker': ['']*20}

In [52]:
for i in range(10):
    df_final['Date'][2*i] = df_all_date['Date'][i]
    df_final['Date'][2*i + 1] = df_all_date['Date'][i]
    df_final['Type'][2*i] = 'Open'
    df_final['Type'][2*i+1] = 'Closed'
    df_final['StockPrice'][2*i] = df_all_date['Open'][i]
    df_final['StockPrice'][2*i + 1] = df_all_date['Close'][i]
    df_final['Ticker'][2*i] = df_all_date['Ticker'][i]
    df_final['Ticker'][2*i + 1] = df_all_date['Ticker'][i]

In [54]:
df_final['StockPrice']

[41.650001525878906,
 40.52000045776367,
 40.18000030517578,
 43.93000030517578,
 41.95000076293945,
 40.33000183105469,
 40.810001373291016,
 41.83000183105469,
 42.040001678466794,
 42.25000152587891,
 42.460001373291014,
 42.67000122070313,
 42.880001068115234,
 39.9900016784668,
 38.604000091552734,
 37.56999969482422,
 38.15999984741211,
 36.619998931884766,
 36.38999938964844,
 33.45000076293945]

In [55]:
def normalize(x):
    """
    Normalize the input to the range between 0 and 1
    """
    x = np.array(x)
    x_min = np.min(x)
    x_max = np.max(x)
    x_normalized = ((x - x_min) / (x_max - x_min)).tolist()
    # append the min and max price in the time window for de-normalization
    x_normalized.extend([float(x_min), float(x_max)])
    return x_normalized

In [56]:
normalize(df_final['StockPrice'])

[0.7824428550680694,
 0.6746183209580132,
 0.6421755568894054,
 1.0,
 0.8110687377173477,
 0.6564886802129679,
 0.7022901652513823,
 0.7996184574572057,
 0.8196566117114849,
 0.8396947659657646,
 0.8597329202200439,
 0.8797710744743237,
 0.8998092287286028,
 0.6240459161443601,
 0.4917938505475802,
 0.3931296862447762,
 0.4494274131874236,
 0.30248075452385625,
 0.2805342323594814,
 0.0,
 33.45000076293945,
 43.93000030517578]

In [60]:
ticker = 'RIOT'
    
enddate = datetime.date.today()
startdate = enddate - datetime.timedelta(days=10)

dates = generate_full_date(startdate, enddate)
df = yf.download(ticker, group_by="Ticker", start=startdate.strftime("%Y-%m-%d"), end=enddate.strftime("%Y-%m-%d"))

[*********************100%***********************]  1 of 1 completed


In [62]:
df['ticker'] = ticker

In [63]:
df['ticker']

Date
2021-04-27    RIOT
2021-04-28    RIOT
2021-04-29    RIOT
2021-04-30    RIOT
2021-05-03    RIOT
2021-05-04    RIOT
2021-05-05    RIOT
2021-05-06    RIOT
Name: ticker, dtype: object

In [65]:
df_final

{'Date': ['2021-04-27',
  '2021-04-27',
  '2021-04-28',
  '2021-04-28',
  '2021-04-29',
  '2021-04-29',
  '2021-04-30',
  '2021-04-30',
  '2021-05-01',
  '2021-05-01',
  '2021-05-02',
  '2021-05-02',
  '2021-05-03',
  '2021-05-03',
  '2021-05-04',
  '2021-05-04',
  '2021-05-05',
  '2021-05-05',
  '2021-05-06',
  '2021-05-06'],
 'Type': ['Open',
  'Closed',
  'Open',
  'Closed',
  'Open',
  'Closed',
  'Open',
  'Closed',
  'Open',
  'Closed',
  'Open',
  'Closed',
  'Open',
  'Closed',
  'Open',
  'Closed',
  'Open',
  'Closed',
  'Open',
  'Closed'],
 'StockPrice': [41.650001525878906,
  40.52000045776367,
  40.18000030517578,
  43.93000030517578,
  41.95000076293945,
  40.33000183105469,
  40.810001373291016,
  41.83000183105469,
  42.040001678466794,
  42.25000152587891,
  42.460001373291014,
  42.67000122070313,
  42.880001068115234,
  39.9900016784668,
  38.604000091552734,
  37.56999969482422,
  38.15999984741211,
  36.619998931884766,
  36.38999938964844,
  33.45000076293945],
 

__Full Script__

In [68]:
import yfinance as yf  # api yfinance to get historical price and big firm recommendation
import numpy as np
import datetime


def generate_full_date(sdate, edate):
    dates = []
    while sdate < edate:
        dates.append(datetime.datetime.strftime(sdate,'%Y-%m-%d'))
        sdate += datetime.timedelta(days=1)
    return dates

def normalize(x):
    """
    Normalize the input to the range between 0 and 1
    """
    x = np.array(x)
    x_min = np.min(x)
    x_max = np.max(x)
    x_normalized = ((x - x_min) / (x_max - x_min)).tolist()
    # append the min and max price in the time window for de-normalization
    x_normalized.extend([float(x_min), float(x_max)])
    return x_normalized

def main():
    ticker = 'RIOT'
    
    enddate = datetime.date.today()
    startdate = enddate - datetime.timedelta(days=10)

    dates = generate_full_date(startdate, enddate)
    df = yf.download(ticker, group_by="Ticker", start=startdate.strftime("%Y-%m-%d"), end=enddate.strftime("%Y-%m-%d"))
    df['ticker'] = ticker
    date_index = df.index.strftime('%Y-%m-%d').tolist()
    df_all_date = {'Date': ['']*10, 'Open': ['']*10, 'Close': ['']*10, 'Ticker': ['']*10}

    for i, date in enumerate(dates):
        df_all_date['Date'][i] = date
        df_all_date['Ticker'][i] = ticker
        if date in date_index:
            df_all_date['Open'][i] = df['Open'][date]
            df_all_date['Close'][i] = df['Close'][date]
        else:
            df_all_date['Open'][i] = 0
            df_all_date['Close'][i] = 0

    for i, date in enumerate(dates):
        if df_all_date['Open'][i] == 0:
            k = 0
            while df_all_date['Open'][i + k] == 0:
                k += 1
            s = df_all_date['Close'][i - 1]
            t = df_all_date['Open'][i + k] - s
            for j in range(k):
                df_all_date['Open'][i + j] = s + t/(2*k+1)*(2*j+1)
                df_all_date['Close'][i + j] = s + t/(2*k+1)*(2*j+2)

    df_final = {'Date': [''] * 20, 'Type': [''] * 20, 'StockPrice': [''] * 20, 'Ticker': [''] * 20}
    for i in range(10):
        df_final['Date'][2 * i] = df_all_date['Date'][i]
        df_final['Date'][2 * i + 1] = df_all_date['Date'][i]
        df_final['Type'][2 * i] = 'Open'
        df_final['Type'][2 * i + 1] = 'Closed'
        df_final['StockPrice'][2 * i] = df_all_date['Open'][i]
        df_final['StockPrice'][2 * i + 1] = df_all_date['Close'][i]
        df_final['Ticker'][2 * i] = ticker
        df_final['Ticker'][2 * i + 1] = ticker
    
    np.savetxt('today_predict.txt', normalize(df_final['StockPrice']))

if __name__ == '__main__':
    main()

[*********************100%***********************]  1 of 1 completed
